In [1]:
# Copyright 2018 Anar Amirli
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Train/Test Set Construction
In this document, we construct training and test sets from already computed feature sets. The sets are computed according to the layout:

data<br>
├── train_features<br>
├── test_features<br>

In [2]:
from os import listdir
from os.path import join
import pickle
import numpy as np
import pandas as pd
import multiprocessing
from collections import Counter
from random import shuffle
import json
import re

# our common utility functions that are used in different notebooks
from utils import *

pd.set_option('compute.use_bottleneck', True)
pd.set_option('compute.use_numexpr', True)

In [3]:
def describe_dataset(dataset):
    print('Shape: {}\n'.format(dataset.shape))
    
    _, col = dataset.shape
        
    heading = 'Pitch index counts'
    print(heading + '\n' + '-'*len(heading))
    for key, val in sorted(Counter(dataset.values[:, col-1]).items()):
        print('{}\t: {}'.format(int(key), val))

## Pitch IDS

In [5]:
if __name__ == '__main__':

    match_id = 60565
    try:
        features_df = pd.read_csv('../data/general/feature-set/match{}_features_2.csv'.format(match_id))
    except FileNotFoundError:
        pass

    ball_idxs = np.sort(features_df['pitch_index'].unique())
    print('Ball  index: {}'.format(ball_idxs))

    print('Size: {}'.format(len(ball_idxs)))

Ball  index: [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9.]
Size: 10


Null index(0) represents all the event categories, apart from the ones we are interested in, that occur when the game stops.

## Dataset Construction

In this section, we construct a combined shuffled dataset for train and set sets, coming from feature data.


### Parameters

When obtaining feature data for a given index, set we should reduce the number of frames of the null ball events in our data set. But cosidering the fact that they are most common, we should keep their numbers close to the number of other most occurred ball location indexes.

* $-$: No-labaled positions intervals would lead to greater label noise.

### Construction
Here we construct the combined dataset from all match data we have in the given data directories.

In [6]:
    # file based constants
    features_dir     = '../data/general/feature-set/'
    features_regex   = re.compile(r'match\d+_features_2.csv')

We speed up the computation by using all the CPU cores via multiprocessing module.

In [7]:
    pool = multiprocessing.Pool()
    df = pd.DataFrame()

    
    features_csv_files = [f for f in listdir(features_dir) if features_regex.match(f)]
    shuffle(features_csv_files)
    print('All matches: {}'.format(len(features_csv_files)))
    features_train_csv = features_csv_files[0:248]
    features_test_csv  = features_csv_files[248:len(features_csv_files)]
    print('Train matches: {}'.format(len(features_train_csv)))
    print('Test matches: {}'.format(len(features_test_csv)))
    
    
    df_train = pd.concat(pool.map(construct_data_set, features_train_csv))
    df_test = pd.concat(pool.map(construct_data_set, features_test_csv))


    print('Train-set shape: {}'.format(df_train.shape))
    print('Test-set shape: {}'.format(df_test.shape))

All matches: 298
Train matches: 248
Test matches: 50
Train-set shape: (563593, 228)
Test-set shape: (115789, 228)


In [8]:
    # shuffling data-sets agin
    # df_train = df_train.drop('homeGoalKeeperY', axis=1)
    
    df_train = df_train.sample(frac=1).reset_index(drop=True)
    df_test = df_test.sample(frac=1).reset_index(drop=True)

### Train Set

In [10]:
    describe_dataset(df_train)
    display(df_train.head())

Shape: (563593, 228)

Pitch index counts
------------------
0	: 175156
1	: 12276
2	: 32470
3	: 11364
4	: 74464
5	: 97929
6	: 75707
7	: 18948
8	: 43623
9	: 21656


,# refX,refY,refSpeed,refDirectX,refDirectY,teamsDBSCANx,teamsDBSCANy,homeDBSCANx,homeDBSCANy,homeMaxX,...,awayCenterForwardAvgY,awayCenterForwardAvgSpeed,awayCenterForwardSlowAvgX,awayCenterForwardSlowAvgY,awayCenterForwardHirAvgX,awayCenterForwardHirAvgY,awayCenterForwardMaxSprintX,awayCenterForwardMaxSprintY,awayCenterForwardMaxSprintSpeed,pitch_index
0,72.250000,26.218750,1.889648,1.0,1.0,79.000000,28.453125,78.062500,43.031250,83.2500,...,21.343750,1.549805,0.00000,0.0000,58.875000,21.34375,0.0000,0.000000,0.000000,5.0
1,52.187500,21.234375,5.558594,1.0,-1.0,41.406250,23.109375,39.218750,17.343750,71.0625,...,23.093750,5.523438,0.00000,0.0000,0.000000,0.00000,66.3125,10.156250,7.511719,4.0
2,48.218750,34.031250,2.640625,-1.0,-1.0,52.625000,23.109375,50.687500,13.921875,68.0625,...,10.351562,3.839844,0.00000,0.0000,0.000000,0.00000,50.7500,10.351562,3.839844,4.0
3,23.015625,32.406250,1.450195,-1.0,1.0,12.015625,25.515625,11.921875,25.937500,52.3125,...,34.000000,2.259766,0.00000,0.0000,11.226562,34.00000,0.0000,0.000000,0.000000,5.0
4,85.062500,40.812500,1.410156,1.0,1.0,92.437500,44.406250,82.000000,45.000000,100.1250,...,44.437500,0.040009,54.53125,44.4375,0.000000,0.00000,0.0000,0.000000,0.000000,0.0


### Test Set

In [12]:
    describe_dataset(df_test)
    display(df_test.head())

Shape: (115789, 228)

Pitch index counts
------------------
0	: 34581
1	: 2467
2	: 6702
3	: 2290
4	: 15888
5	: 20966
6	: 15862
7	: 3688
8	: 8882
9	: 4463


,# refX,refY,refSpeed,refDirectX,refDirectY,teamsDBSCANx,teamsDBSCANy,homeDBSCANx,homeDBSCANy,homeMaxX,...,awayCenterForwardAvgY,awayCenterForwardAvgSpeed,awayCenterForwardSlowAvgX,awayCenterForwardSlowAvgY,awayCenterForwardHirAvgX,awayCenterForwardHirAvgY,awayCenterForwardMaxSprintX,awayCenterForwardMaxSprintY,awayCenterForwardMaxSprintSpeed,pitch_index
0,23.203125,51.28125,0.930176,-1.0,1.0,17.640625,42.50000,18.0625,42.56250,48.56250,...,34.34375,1.519531,0.00000,0.00000,13.148438,34.34375,0.0000,0.0000,0.000000,6.0
1,86.000000,48.87500,0.119995,-1.0,-1.0,86.937500,39.59375,84.1250,38.90625,89.62500,...,34.93750,1.584961,84.31250,32.84375,90.625000,37.06250,0.0000,0.0000,0.000000,0.0
2,70.687500,37.71875,1.480469,1.0,1.0,73.625000,42.03125,71.5625,35.96875,79.43750,...,42.12500,2.189453,0.00000,0.00000,59.812500,42.12500,0.0000,0.0000,0.000000,6.0
3,67.937500,41.65625,1.190430,-1.0,-1.0,66.812500,42.78125,65.8750,34.59375,74.37500,...,46.18750,5.039062,0.00000,0.00000,0.000000,0.00000,51.7500,46.1875,5.039062,5.0
4,49.343750,52.53125,1.440430,-1.0,1.0,44.656250,46.40625,41.4375,47.21875,61.90625,...,56.00000,2.289062,46.84375,50.37500,0.000000,0.00000,39.8125,61.6250,3.539062,0.0


## Exporting

In [13]:
    df_train.to_csv('../data/general/{dataset}/all_{dataset}_3.csv'.format(dataset='train'), index=False)
    df_test.to_csv('../data/general/{dataset}/all_{dataset}_3.csv'.format(dataset='test'), index=False)

In [14]:
    df_train[1:10]

,# refX,refY,refSpeed,refDirectX,refDirectY,teamsDBSCANx,teamsDBSCANy,homeDBSCANx,homeDBSCANy,homeMaxX,...,awayCenterForwardAvgY,awayCenterForwardAvgSpeed,awayCenterForwardSlowAvgX,awayCenterForwardSlowAvgY,awayCenterForwardHirAvgX,awayCenterForwardHirAvgY,awayCenterForwardMaxSprintX,awayCenterForwardMaxSprintY,awayCenterForwardMaxSprintSpeed,pitch_index
1,52.187500,21.234375,5.558594,1.0,-1.0,41.406250,23.109375,39.218750,17.343750,71.0625,...,23.093750,5.523438,0.000000,0.000000,0.000000,0.00000,66.3125,10.156250,7.511719,4.0
2,48.218750,34.031250,2.640625,-1.0,-1.0,52.625000,23.109375,50.687500,13.921875,68.0625,...,10.351562,3.839844,0.000000,0.000000,0.000000,0.00000,50.7500,10.351562,3.839844,4.0
3,23.015625,32.406250,1.450195,-1.0,1.0,12.015625,25.515625,11.921875,25.937500,52.3125,...,34.000000,2.259766,0.000000,0.000000,11.226562,34.00000,0.0000,0.000000,0.000000,5.0
4,85.062500,40.812500,1.410156,1.0,1.0,92.437500,44.406250,82.000000,45.000000,100.1250,...,44.437500,0.040009,54.531250,44.437500,0.000000,0.00000,0.0000,0.000000,0.000000,0.0
5,77.562500,23.515625,1.110352,1.0,-1.0,83.375000,32.562500,80.562500,29.828125,87.8750,...,47.125000,1.344727,68.187500,47.125000,0.000000,0.00000,0.0000,0.000000,0.000000,8.0
6,36.156250,25.296875,0.939941,-1.0,-1.0,40.750000,20.437500,33.437500,20.218750,56.6875,...,25.234375,0.930176,18.953125,25.234375,0.000000,0.00000,0.0000,0.000000,0.000000,2.0
7,36.625000,29.656250,3.609375,1.0,-1.0,36.031250,29.859375,22.390625,31.890625,58.3750,...,34.468750,2.669922,0.000000,0.000000,17.046875,34.46875,0.0000,0.000000,0.000000,1.0
8,82.437500,43.156250,0.370117,-1.0,-1.0,92.750000,44.968750,88.812500,51.375000,101.9375,...,57.750000,1.540039,0.000000,0.000000,77.125000,57.75000,0.0000,0.000000,0.000000,0.0
9,63.187500,27.062500,1.669922,1.0,-1.0,63.625000,34.562500,56.343750,33.687500,72.4375,...,30.906250,3.449219,0.000000,0.000000,55.562500,40.15625,56.4375,21.625000,3.919922,5.0
